**ドライブ共有準備**

In [0]:
from google.colab import drive
drive.mount('./gdrive')



---



# scikit-learn(sklearn)でBostonデータセットを重回帰分析してみる
*   scikit-learnは初心者向けの機械学習のオープンソースライブラリ。本家は[ここ](https://scikit-learn.org/stable/user_guide.html)
*   [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) で公開されている、[ボストン市郊外の地域別住宅価格](
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/)を基に、重回帰によって住宅価格(目的変数)を予測した例。
*   参考になるサイトは、[ここ](https://pythondatascience.plavox.info/scikit-learn/%E7%B7%9A%E5%BD%A2%E5%9B%9E%E5%B8%B0)とか。

### データ準備

In [0]:
import pandas as pd    # 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Bostonデータセットをロード
boston = datasets.load_boston()    # 変数に代入
#print(boston)
#print(boston.DESCR)    # データセットの説明文
#print(boston.data)
#print(boston.feature_names)
#print(boston.target)

# set df
boston_df=pd.DataFrame(boston.data)    # データフレームの引数にboston.dataを入れる
boston_df.columns = boston.feature_names    # カラムに先程のboston.feature_namesを指定
boston_df['PRICE'] = pd.DataFrame(boston.target)   # 住宅価格データ（目的変数）の追加
# 参考）CSV出力
#boston_df.to_csv('boston_df.csv', sep=',', index=True, encoding='utf-8')
#boston_df.to_csv('boston.csv', sep=',', index=False, encoding='utf-8')

In [0]:
boston_df.head(5)    # 中身の確認（先頭5行）

In [0]:
#print(boston.DESCR)    # データ項目の説明（参考）

【参考】Boston House-price（ボストン市の住宅価格）データサンプルのカラム説明（レコード数は506） 

**CRIM**:人口 1 人当たりの犯罪発生数、**ZN**:25,000 平方フィート以上の住居区画の占める割合、**INDUS**:小売業以外の商業が占める面積の割合、**CHAS**:チャールズ川によるダミー変数 (1: 川の周辺, 0: それ以外)、**NOX**:NOxの濃度、**RM**:住居の平均部屋数、**AGE**:1940 年より前に建てられた物件の割合、**DIS**:5 つのボストン市の雇用施設からの距離 (重み付け済)、**RAD**:環状高速道路へのアクセスしやすさ、**TAX**:$10,000ドルあたりの不動産税率の総計、**PTRATIO**:町毎の児童と教師の比率、**B**:町毎の黒人(Bk)の比率を次の式で表したもの→1000(Bk-0.63)^2、**LSTA**:給与の低い職業に従事する人口の割合(%)


### 重回帰分析

**正規化（標準化）**  ：StandardScalerクラスで標準化した説明変数X2を準備

正規化は、データの平均値からのばらつき（偏差）を、ばらつきの平均値（標準偏差）で割ってやることで、項目間のスケールが違っても平等に評価できるようにする処理（らしい。標準化とも呼ばれる）。

In [0]:
# Ｘの準備（基データ）
X = boston_df.drop("PRICE", axis=1)    # PRICEのカラムを削除
print("------X.describe()の表示------")
print("countは件数、meanは平均、stdは標準偏差、25%は第一四分位数　などなど")
print(X.describe())

# Ｘ２の準備：ＸをsktlearnのStandardScalerクラスで標準化する
scaler = StandardScaler()
scaler.fit(X)       #  fit():渡されたデータの最大値、最小値、平均、標準偏差、傾き...などの統計を取得して、内部メモリに保存する
#print("scaler.fit(X) ->" + str(scaler.fit(X)))
#print("scaler.mean_ ->" + str(scaler.mean_))
#print("scaler.transform ->" + str(scaler.transform(X)))
X2 = scaler.transform(X)     # transform:fit()で取得した統計情報を使って、渡されたデータを実際に書き換える
X2 = pd.DataFrame(X2)
X2.columns = boston.feature_names
#print("X2.describe() ->")
#print(X2.describe())

# Ｙの準備
Y = boston_df.PRICE 

In [0]:
X.head(5)   # 素のX（先頭5行）

In [0]:
X2.head(5)   # StandardScalerクラスで標準化したあと（先頭5行）

In [0]:
Y.head(5)   #Ｙ（先頭5行）

**回帰分析**  ：LinearRegressionクラスで説明変数から目的変数を予測

In [0]:
# Xでの重回帰分析
#clf = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
#clf.fit(X, Y)
# Xを標準化したデータX2での重回帰分析（予測モデルの作成）
clf2 = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
clf2.fit(X2, Y)

# 入力がX(標準化前)
#print(pd.DataFrame({"Name":X.columns,
#                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') ) # 係数でソート
#print(clf.intercept_) # 切片
#print(clf.score(X,Y)) # R^2

# 入力がX2(標準化後)
print("どのカラムが説明変数に大きい影響を与えているのかを、係数で確認！")
print(pd.DataFrame({"Name":X2.columns,
                    "Coefficients":clf2.coef_}).sort_values(by='Coefficients') )    # coef_ ：偏回帰係数
print(clf2.intercept_)    # intercept_ ：切片
print(clf2.score(X2,Y))   # score ：決定係数 R2を出力

【再掲】カラム説明　※値が大きいカラムほど関係性が高い

**CRIM**:人口 1 人当たりの犯罪発生数、**ZN**:25,000 平方フィート以上の住居区画の占める割合、**INDUS**:小売業以外の商業が占める面積の割合、**CHAS**:チャールズ川によるダミー変数 (1: 川の周辺, 0: それ以外)、**NOX**:NOxの濃度、[**RM:住居の平均部屋数**](https://)、**AGE**:1940 年より前に建てられた物件の割合、**DIS**:5 つのボストン市の雇用施設からの距離 (重み付け済)、[**RAD:環状高速道路へのアクセスしやすさ**](https://)、**TAX**:$10,000ドルあたりの不動産税率の総計、**PTRATIO**:町毎の児童と教師の比率、**B**:町毎の黒人(Bk)の比率を次の式で表したもの→1000(Bk-0.63)^2、**LSTA**:給与の低い職業に従事する人口の割合(%)


因みに式で表すと次の通り。

[PRICE] = -3.743627 * [LSTAT] + -3.104044 * [DIS] + -2.076782 * [TAX] + -2.060607 * [PTRATIO] + -2.056718 * [NOX] + -0.928146 * [CRIM] + 0.019466 * [AGE] + 0.140900 * [INDUS] + 0.849268 * [B] + 1.081569 * [ZN] + 2.662218 * [RAD] + 2.674230 * [RM] + 22.532806324110677     ※最後は切片

In [0]:
# inputがXの結果
#plt.figure(1)
#plt.title('raw X-data')
#plt.xlabel('RM (number of rooms)', fontsize=14)
#plt.ylabel('PRICE (target)', fontsize=14)
#plt.scatter(X["RM"], Y, c='blue', label='Raw data')
#plt.scatter(X.RM, clf.predict(X), c='red', label='Multiple regression analysis')
#plt.legend(loc='lower right', fontsize=12)
#plt.show()

# inputがX2の結果
plt.figure(2)
plt.title('Standardization of X-data')
plt.xlabel('RM (number of rooms)', fontsize=14)
plt.ylabel('PRICE (target)', fontsize=14)
plt.scatter(X2.RM, Y, c='blue', label='Raw data')   # ブルーはRowデータを散布図(scatter)にマッピング
plt.scatter(X2.RM, clf2.predict(X2), c='red', label='Multiple regression analysis')   # model.predict(x)：モデルを使用して、xに対して予測を実行し予測値を算出
plt.legend(loc='lower right', fontsize=14)
plt.show()

### データアップロード ※今回は使わない

In [0]:
# Google DriveからColabへのファイルコピーする方法
!!pip install -U -q PyDrive

In [0]:
# Google Driveアクセス準備
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Google Driveからのコピー
id = '1icpF-Z7x202u_6AOxjRsDmTk_gy7oZNw'  # 共有リンクで取得した id= より後の部分(mocha.zipの例)
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mocha.zip')
id = '1-uXHH1VAZim-YuCrWl-aO9B2AM6LrTB5'  # 共有リンクで取得した id= より後の部分(kuru.zipの例)
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('kuru.zip')
id = '1L2WozAd2ABZU39p8fKVStGbyqZTgrxnA'  # 共有リンクで取得した id= より後の部分(otrdog.zipの例)
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('otrdog.zip')
# 終わったら、ファイル画面の更新をして確認

In [0]:
# Zipの場合は展開
!unzip mocha.zip
!unzip kuru.zip
!unzip otrdog.zip

### （その他）メモ書きなど

---